In [0]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
import numpy as np
import math
import matplotlib.pyplot as plt
from copy import deepcopy
from operator import attrgetter
import random

In [69]:
from google.colab import files
uploaded = files.upload()

Saving allDataReformated.csv to allDataReformated (1).csv


In [0]:
num_cities = 16
features = []
trains = []

header = []
for c in range(num_cities):
	header.append(f"c{c}x")
	header.append(f"c{c}y")
	features.append(f"c{c}x")
	features.append(f"c{c}y")

for c in range(1, num_cities):
	header.append(f"o{c}")
	trains.append(f"o{c}")

header.append("total")


all_data = pd.read_csv("allDataReformated.csv", names = header)


In [0]:
X_train_ori, X_test_ori, y_train, y_test = train_test_split(all_data.loc[:,features],all_data.loc[:,trains])
#y_train.head()

In [0]:
#scaling
scaler = StandardScaler()
scaler.fit(X_train_ori)

X_train = scaler.transform(X_train_ori)
X_test = scaler.transform(X_test_ori)

In [73]:
mlp = MLPRegressor(hidden_layer_sizes=(20,20),max_iter=10000)

mlp.fit(X_train,y_train)

MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(20, 20), learning_rate='constant',
       learning_rate_init=0.001, max_iter=10000, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [74]:
predictions = mlp.predict(X_test)
predictions


array([[164.46659718, 164.22202601, 171.49787974, ..., 165.18290019,
        174.87214495, 174.14576909],
       [151.58513774, 152.16473484, 152.30320883, ..., 147.45537053,
        158.40660998, 159.32382678],
       [168.17469519, 165.46239558, 172.25978503, ..., 170.33472114,
        174.29419862, 183.81556363],
       ...,
       [184.73758155, 182.77155577, 196.20086422, ..., 190.73105314,
        194.22392533, 196.89019444],
       [175.45745491, 171.60248023, 180.88251366, ..., 175.99169413,
        181.19387333, 186.14137053],
       [199.12343939, 203.17428794, 204.50329531, ..., 192.99222667,
        212.72659483, 211.54561077]])

In [75]:
class city:
	name = "_";
	x = 0
	y = 0
  
def distance(city1, city2):
    return math.sqrt((city1.x - city2.x)**2 + (city1.y - city2.y)**2)


print(predictions.shape,X_test_ori.shape)

(337, 15) (337, 32)


In [0]:
print(X_test_ori.values)

[[55 46 96 ... 95 31  3]
 [73 11 74 ... 60 92 83]
 [ 7 16 99 ... 93 59 44]
 ...
 [61 63 53 ... 96 48  1]
 [44 80 68 ... 26 91 69]
 [22 73 21 ...  2  9  1]]


In [76]:

def getError(train,actual,predict):
  total_error = 0

  for test_line in range(predict.shape[0]):
    cities = []
    for c in range(num_cities):
      new_city = city()
      new_city.name = c
      new_city.x = int(train[test_line,c*2])
      new_city.y = int(train[test_line,c*2 + 1])
      cities.append(new_city)
    #print(predict)
    #print(actual[test_line])
    actual_order = np.argsort(actual[test_line])
    predict_order = np.argsort(predict[test_line])
    
    actual_path_distance = 0
    predict_path_distance = 0
    
    actual_last_city = cities[0]
    predict_last_city = cities[0]
    #print(actual_order)
    for c in range(num_cities - 1):
      #print(actual_last_city.name)
      actual_path_distance += distance(actual_last_city,cities[actual_order[c]+1])
      predict_path_distance += distance(predict_last_city,cities[predict_order[c]+1])
      #print(actual_path_distance)
      
      actual_last_city = cities[actual_order[c]+1]
      predict_last_city = cities[predict_order[c]+1]
      
    actual_path_distance += distance(cities[0],actual_last_city)
    predict_path_distance += distance(cities[0],predict_last_city)
    total_error += (predict_path_distance-actual_path_distance)
    #print(actual_path_distance,predict_path_distance)
  return total_error/predict.shape[0]
    #return
    
#getError(X_test_ori.values,y_test.values,predictions)
#print(X_test_ori.values[0], y_test.values[0])
train_predict = mlp.predict(X_train)
first_err = getError(X_train_ori.values,y_train.values,train_predict)
#print(getError(X_train_ori.values,y_train.values,train_predict))
print(first_err)

489.17682398905214


In [77]:
class modelPreformer:
  model = None
  err = -1
  def __init__(self,model,score):
    self.model = model
    self.err = score
  
sample_size = 100
models = []

for s in range(sample_size):
  new_model = fullyMutate(deepcopy(mlp))
  train_evaluate = new_model.predict(X_train)
  new_err = getError(X_train_ori.values,y_train.values,train_evaluate)
  print(new_err)
  models.append(modelPreformer(new_model,new_err))
  


487.46243747858676
490.84390275632467
489.4606121446761
484.67624185101005
487.16011820296245
486.60283083266125
484.88379099551565
488.1406575473384
490.541147849721
485.2160574821233
484.39744868828205
489.2336282392048
485.2720054338723
481.50196624137436
487.3360616924448
483.11949749362344
486.72421318073765
484.0261018679171
484.47505938755853
486.9274939951277
486.4099705162716
484.58694567857503
485.1549509088946
483.82567492171404
493.76420059869724
479.2304824896107
484.33056460018116
484.1401670107557
485.8355765974111
486.16192240535736
484.84714407682776
490.7552127557271
485.0689313156033
485.0236228392322
487.86650133887724
488.92115924908427
488.02911463858476
487.5457181307305
484.04903067130044
488.3443086434011
492.3322729135095
489.5899215808419
485.3252479796058
487.65114185861995
487.2162112928829
489.5463100227664
490.8906302549856
485.3849360956781
491.9922754795007
486.1895327414447
483.4548234797944
488.01199948267197
482.89817166300907
488.9786396151198
485.8

In [0]:
mutation_rate = 0.1
def mutation(model):
  for layer_num in range(len(model.coefs_)):
    for height_num in range(len(model.coefs_[layer_num])):
      for weight_num in range(len(model.coefs_[layer_num][height_num])):
        if random.random() < mutation_rate:
          model.coefs_[layer_num][height_num][weight_num] = random.random()*2 - 1
  return model
          
def crossover(parent1,parent2):
  child = deepcopy(parent1)
  for layer_num in range(len(parent1.coefs_)):
    for height_num in range(len(parent1.coefs_[layer_num])):
      for weight_num in range(len(parent1.coefs_[layer_num][height_num])):
        if random.random() < 0.5:
          child.coefs_[layer_num][height_num][weight_num] = parent2.coefs_[layer_num][height_num][weight_num]
  return child
          
def fullyMutate(model):
  for layer_num in range(len(model.coefs_)):
    for height_num in range(len(model.coefs_[layer_num])):
      for weight_num in range(len(model.coefs_[layer_num][height_num])):
        model.coefs_[layer_num][height_num][weight_num] = random.random()*2 - 1
  return model
        

In [79]:

num_generations = 100
generation = 0
while generation < num_generations:
  models = sorted(models,key=attrgetter('err'))
  print(models[0].err)
  generation+=1
  parents = int(sample_size/10)
  for i in range(parents,int(sample_size)):
    child_model = crossover(random.choice(models[:parents]).model,random.choice(models[:parents]).model)
    #print(child_model)
    child_model = mutation(child_model)
    train_evaluate = child_model.predict(X_train)
    models[i] = modelPreformer(child_model,getError(X_train_ori.values,y_train.values,train_evaluate))
    
    
    

    

479.2304824896107
478.16819107760097
478.16819107760097
478.16819107760097
478.16819107760097
478.16819107760097
478.16819107760097
478.16819107760097
478.16819107760097
478.03446476904713
478.03446476904713
478.03446476904713
478.03446476904713
478.03446476904713
478.03446476904713
478.03446476904713
478.03446476904713
478.03446476904713
478.03446476904713
478.03446476904713
478.03446476904713
478.03446476904713
478.03446476904713
478.03446476904713
478.03446476904713
478.03446476904713
478.03446476904713
478.03446476904713
478.03446476904713
478.03446476904713
478.03446476904713
478.03446476904713
478.03446476904713
478.03446476904713
478.03446476904713
478.03446476904713
478.03446476904713
477.6756406465389
476.75918421135185
476.75918421135185
476.75918421135185
476.75918421135185
476.75918421135185
476.75918421135185
476.75918421135185
476.75918421135185
476.75918421135185
476.75918421135185
476.75918421135185
476.75918421135185
476.75918421135185
476.75918421135185
476.7591842113

In [0]:
cities = []
for c in range(num_cities):
  new_city = city()
  new_city.name = c
  new_city.x = int(X_train_ori.values[3,c*2])
  new_city.y = int(X_train_ori.values[3,c*2 + 1])
  cities.append(new_city)

print(np.argsort(y_train.values[3]),np.argsort(trainTrack[3]))
#X_train_ori.head()
d=[0]*16
d[0] = distance(cities[0],cities[8])
d[1] = distance(cities[8],cities[1])
d[2] = distance(cities[1],cities[2])
d[3] = distance(cities[2],cities[10])
d[4] = distance(cities[10],cities[6])
d[5] = distance(cities[6],cities[15])
d[6] = distance(cities[15],cities[7])
d[7] = distance(cities[7],cities[4])
d[8] = distance(cities[4],cities[14])
d[9] = distance(cities[14],cities[11])
d[10] = distance(cities[11],cities[9])
d[11] = distance(cities[9],cities[13])
d[12] = distance(cities[13],cities[3])
d[13] = distance(cities[3],cities[5])
d[14] = distance(cities[5],cities[12])
d[15] = distance(cities[12],cities[0])

print(d)
print(sum(d))

In [0]:
trainTrack = mlp.predict(X_train)
getError(X_train_ori.values,y_train.values,trainTrack)


In [33]:
#fig, axes = plt.subplots(4, 4)
#print(mlp.coefs_)
#print(len(mlp.coefs_))
#print(len(mlp.coefs_[0]))
#print(len(mlp.coefs_[1]))
#mlp.coefs_[0][0][0] = -1
#print(mlp.coefs_[0][0][0])

def shape(arr, d):
  try:
    print(d,len(arr))
  except:
    return
  for i in arr:
    shape(i, d+1)
    
    
shape(mlp.coefs_,0)

#vmin, vmax = mlp.coefs_[0].min(), mlp.coefs_[0].max()
#for coef, ax in zip(mlp.coefs_[0].T, axes.ravel()):
#    ax.matshow(coef.reshape(32), cmap=plt.cm.gray, vmin=.5 * vmin,
#               vmax=.5 * vmax)
#    ax.set_xticks(())
#    ax.set_yticks(())

#plt.show()

0 4
1 32
2 30
2 30
2 30
2 30
2 30
2 30
2 30
2 30
2 30
2 30
2 30
2 30
2 30
2 30
2 30
2 30
2 30
2 30
2 30
2 30
2 30
2 30
2 30
2 30
2 30
2 30
2 30
2 30
2 30
2 30
2 30
2 30
1 30
2 35
2 35
2 35
2 35
2 35
2 35
2 35
2 35
2 35
2 35
2 35
2 35
2 35
2 35
2 35
2 35
2 35
2 35
2 35
2 35
2 35
2 35
2 35
2 35
2 35
2 35
2 35
2 35
2 35
2 35
1 35
2 20
2 20
2 20
2 20
2 20
2 20
2 20
2 20
2 20
2 20
2 20
2 20
2 20
2 20
2 20
2 20
2 20
2 20
2 20
2 20
2 20
2 20
2 20
2 20
2 20
2 20
2 20
2 20
2 20
2 20
2 20
2 20
2 20
2 20
2 20
1 20
2 15
2 15
2 15
2 15
2 15
2 15
2 15
2 15
2 15
2 15
2 15
2 15
2 15
2 15
2 15
2 15
2 15
2 15
2 15
2 15
